In [ ]:
import os
import string
import gensim
import re

In [ ]:
# a memory-friendly iterator
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                print(os.path.join(self.dirname, fname))
                yield line.split()

In [ ]:
# read data
root = '' # root of your data 
polishReviews = open(root, 'r',errors='ignore',encoding="utf8")
lines = polishReviews.readlines()

In [ ]:
#data cleaning and preprocessing. Some variants: 
# lower
# lower + punctuation cleaning
# changing near words 
#----------------------------
posReviews = ''
negReviews =''
mostCommonWords = ['i','w','z', 'na', 'je', 'że', 'do', 'ale', 'mnie','ten', 'tego', 'tak', 'kto','wiem','ja', 'od','fil', 'któr','który']
for line in lines:
    line = line.lower()
    newLine = ' '.join(line.split())
    newLine = newLine.replace('"','')
    review = newLine[:(len(newLine) - 2)]
    gradeLine = newLine[(len(newLine) - 2):]
    if ',' in gradeLine:
        gradeLine = gradeLine.replace(',','')
    if int(gradeLine) > 5:
        review = review.translate(str.maketrans('','',string.punctuation)).split()        
        review = ' '.join(review)
        posReviews += review
        posReviews += '\n'

    else:
        review = review.translate(str.maketrans('','',string.punctuation)).split()
        review = ' '.join(review)
        negReviews += review
        negReviews += '\n'
#posReviewsSplitted = posReviews.split()
#negReviewsSplitted = negReviews.split()
        
for i in mostCommonWords:
    if str(i) in posReviewsSplitted:
        posReviewsSplitted.remove(str(i))
    if str(i) in negReviewsSplitted:
        negReviewsSplitted.remove(str(i))
    
#cleanedPos = ' '.join(posReviewsSplitted)
#cleanedNeg = ' '.join(negReviewsSplitted)
#print(cleanedPos)
for i in hapaxLegomena:
    if str(i) in posReviewsSplitted:
        posReviewsSplitted.remove(str(i))
    if str(i) in negReviewsSplitted:
        negReviewsSplitted.remove(str(i))
posResult = open(r'C:\Users\Michał\Desktop\iet\2_semestr\PJN\2\allPosReviewsLowerWithoutPunctuationWithoutHapaxWithoutCommon.txt', 'w',errors='ignore',encoding="utf8")
posResult.write(posReviews)
posResult.close()
negResult = open(r'C:\Users\Michał\Desktop\iet\2_semestr\PJN\2\allNegReviewsLowerWithoutPunctuationWithoutHapaxWithoutCommon.txt', 'w',errors='ignore',encoding="utf8")
negResult.write(negReviews)
negResult.close()

In [ ]:
# make set of mostCommonWords and HapaXLegomena
from collections import Counter
wholeData = posReviews+'\n'+negReviews
countWords = Counter(wholeData.split())
hapaxLegomena = []
totalWords = len(countWords)
sortedWords = countWords.most_common(totalWords)
for i in sortedWords:
    if i[1] == 1:
        hapaxLegomena.append(i[0])
    
words = ''
counter = 1
for word in sortedWords:
    counter += 1
    if counter == 200:
        break
    words += word[0] +'\n'
#mostCommonWords.write(words)
#mostCommonWords.close()        

In [ ]:
root = '' #root to data to process
file = open(root,mode='r', errors='ignore',encoding="utf8")
lines = file.readlines()
whole = ''
for line in lines:
    whole += line.replace('ą','a').replace('ć','c').replace('ę','e').replace('ń','n').replace('ł','l').replace('ó','o').replace('ś','s').replace('ź','z').replace('ż','z').replace('\n','')+'\n'    
file = open(root,mode='w', errors='ignore',encoding="utf8")   
file.write(whole)
file.close()

In [ ]:
# create json from each line (each line is a review)
root =''
file = open(root ,errors='ignore',encoding="utf8")
lines = file.readlines()
wholeJson =''

for line in lines:    
    json ='{"text":'
    body = '['
    for word in line.split():
        body += '"' + word +'",'
    body = body[0:-1]
    body += ']'
    json += body
    json += ', "label": "neg"}\n'
    wholeJson += json

In [ ]:
result = open(root, 'w',errors='ignore',encoding="utf8")
result.write(wholeJson)
result.close()

In [ ]:
# creation of word2vect model

model = gensim.models.Word2Vec(size = 100, window = 5)
model.build_vocab(reviews)
model.save("polishWord2VectModel.model")

In [ ]:
# prepraing vectors to torxtext format

word2vect = ''

for item in model.wv.vocab:
    stri = str(item)+str(' ')+str(model.wv[item]).replace('[','').replace(']','').replace('\n','')
    stri = re.sub(' +', ' ', stri)
    if len(stri.split()) > 101:
        print(str(item))
    word2vect += stri+'\n'
resFile = open(r'./polishVectors.txt', 'w',errors='ignore',encoding="utf8")    
resFile.write(word2vect)
resFile.close

In [ ]:
root = r'./PolishWithoutPunctuationWithoutHapaxWihoutPolish\\PosReviewsLowerWithoutPunctuationWithoutHapaxLegomenaPreparedWithoutPolishCharactersPrepared.json'
file = open(root, 'r',errors='ignore',encoding="utf8") 
lines = file.readlines()
whole = ''
for line in lines:
    line = line.replace('}','}\n')
    whole += line
print(whole)
file = open(root, 'w',errors='ignore',encoding="utf8") 
file.write(whole)
file.close()

In [ ]:
# json checker - check if the json is in proper format
import json 
polishReviews = open(root, 'r', errors='ignore', encoding="utf8")
lines = polishReviews.readlines()
counter = 0
for line in lines:
    counter += 1
    print(counter)
    json.loads(line)